In [ ]:
# This cell downloads a dataset needed for a demo to a local filesystem and used onedata-fs to copy it onto a space.
# This cell should be run once before a demo as a data preparation job.
import sys, os, fs, subprocess
from fs_onedatafs import OnedataFS
from fs.copy import copy_fs
from fs.osfs import OSFS

# Setup onedata-fs
sys.path.append('/opt/oneclient/lib')
os.listdir("/")

# Get data from my dropbox account
subprocess.call(['wget', '-O', 'data.tgz', 'https://www.dropbox.com/s/2toa3ymec6alrr8/oxford-flower-dataset.tgz?dl=1'])
os.listdir(".")
subprocess.call(['tar', 'zxvf', 'data.tgz'])
os.listdir(".")
dataset="oxford-flower-dataset"
os.listdir(dataset)

# Get configuration from the environment
spaceName=os.environ['SPACE_NAME']
token=os.environ['ONECLIENT_ACCESS_TOKEN']
provider=os.environ['ONECLIENT_PROVIDER_HOST']
insecure=os.environ['ONECLIENT_INSECURE']

# Connect to Oneprovider
odfs = OnedataFS(provider, token, insecure=True, force_proxy_io=True)
space = odfs.opendir('/{}'.format(spaceName))

# Copy data to a space
current_directory_fs = OSFS("./oxford-flower-dataset")
root_target_dir = space.makedir('oxford-flower-dataset')
copy_fs(current_directory_fs, root_target_dir)
root_target_dir.listdir('.')

In [ ]:
# Initialize OnedataFS plugin 
import sys
import matplotlib.pyplot as plt
from PIL import Image
import io
import os
sys.path.append('/opt/oneclient/lib')
from fs_onedatafs import OnedataFS

In [ ]:
# Get configuration from the environment
spaceName=os.environ['SPACE_NAME']
token=os.environ['ONECLIENT_ACCESS_TOKEN']
provider=os.environ['ONECLIENT_PROVIDER_HOST']
insecure=os.environ['ONECLIENT_INSECURE']

print("Space name: {}".format(space))
print("Authentication token: {}".format(token))
print("Oneprovider address: {}".format(provider))

In [ ]:
# Connect to Oneprovider
odfs = OnedataFS(provider, token, insecure=True, force_proxy_io=True)

In [ ]:
# Open selected space DemoWeek and list its contents
space = odfs.opendir('/{}'.format(spaceName))
space.listdir('/')

In [ ]:
# Open and display single image
i000 = space.opendir('oxford-flower-dataset/')
i1 = i000.openbin('image_0001.jpg', 'r')

In [ ]:
# Show file blocks at Oneprovider (should be empty - not replicated yet)
space.location('oxford-flower-dataset/image_0001.jpg') 

In [ ]:
# Create instance of Pillow Image from image bytes
im1 = Image.open(i1)
plt.imshow(im1) 

In [ ]:
# Show file blocks (should be completely replicated)
space.location('oxford-flower-dataset/image_0001.jpg')

In [ ]:
# Close the image file
i1.close() 

In [ ]:
# Generate list of a subset of images paths
image_paths = [img.path for img in space.glob("**/oxford-flower-dataset/*.jpg")]
image_paths

In [ ]:
# Open selected images as Pillow Image objects 
images = [Image.open(space.openbin(path)) for path in image_paths]
images[0]
len(images)

In [ ]:
# Create target thumbnail composite image
result_size = 1000
result = Image.new("RGB", (result_size, result_size))

In [ ]:
# Paste thumbnails of images into target image
i = 0
j = 0
for img in images: 
    img.thumbnail((100, 100), Image.ANTIALIAS)
    w, h = img.size
    print('Adding thumbnail at {0},{1}'.format(i, j, w, h))
    result.paste(img, (i, j, i + w, j + h))
    i += 100
    if i >= result_size:
        i = 0
        j += 100

In [ ]:
# Show thumbnail image
plt.imshow(result)

In [ ]:
# Close source images
[img.close() for img in images]

In [ ]:
# Write result to file in DemoWeek space
space.create("thumbnails.png")
fresult = space.openbin("thumbnails.png", "rw+")
result_bytes = io.BytesIO()
result.save(result_bytes, format='PNG')
fresult.write(result_bytes.getvalue())
fresult.close()